In [ ]:
!brew install minikube     # install kubernetes
!conda install -y requests # for http requests

In [ ]:
# Install Docker Desktop
!brew install --cask docker

In [ ]:
# Start Kubernetes cluster on Docker
!minikube start --driver=docker --cpus=8 --memory=16g

In [ ]:
#Verify cluster
!kubectl get nodes

In [ ]:
# Enable Metrics Server
!minikube addons enable metrics-server

In [ ]:
# In Jupyter notebook, make to run everything in a single shell session. Hence chaining of commands
# Otherwise, kubectl will not find the images that were built in mikube docker env.
!eval $(minikube docker-env) && \
docker build -t llm-sharding:latest . && \
docker images | grep llm-sharding && \
kubectl apply -f k8s/ && \
kubectl get pods

In [ ]:
!kubectl get pods -o wide
!kubectl get svc

In [ ]:
# Run this command on terminal before executing below code
# kubectl port-forward svc/llm-coordinator 5000:5000

import requests

# Define the coordinator service URL
COORDINATOR_URL = "http://127.0.0.1:5000/generate"

# Function to send a prompt and get output
def generate_text(prompt: str, max_new_tokens: int = 20):
    payload = {"prompt": prompt, "max_new_tokens": max_new_tokens}
    try:
        response = requests.post(COORDINATOR_URL, json=payload)
        if response.status_code == 200:
            return response.json()  # Expect JSON output from coordinator
        else:
            return f"Error: {response.status_code} - {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

# Example usage
prompt = "Hi, "
output = generate_text(prompt, max_new_tokens=2)
print("Prompt:", prompt)
print("Generated output:", output)